In [ ]:
import sys
sys.path.append('/orions4-zfs/projects/lins2/Lin_Space/Git_Repos')
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 2
setup_one_gpu(GPU)

In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.stats import hmean

from general_tools.notebook.tf import reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, points_extension, \
                                                 shape_net_core_synth_id_to_category

from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch

from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs, accuracy_of_completion, \
                                           coverage_of_completion, save_reconstructions, \
                                           save_pc_prediction_stats
                                           
                                                  
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud


from tf_lab.autopredictors.scripts.virt_scan_data import load_partial_pointclouds
import tf_lab.autopredictors.scripts.virt_scan_data as vscan
from general_tools.simpletons import indices_in_iterable

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
class_type = raw_input('Give me the class type: ').lower()
class_syn_id = shape_net_category_to_synth_id()[class_type]

loss = 'chamfer'
incomplete_n_samples = 2048
complete_n_samples = 4096
max_training_epochs = 100

do_training = False
load_model = True
do_evaluation = False
do_detailed_plotting = False

#experiment_id = int(raw_input('Give the experiment ID: '))
experiment_tag = raw_input('Give an experiment tag-ID')

experiment_name = class_type + '_' + str(incomplete_n_samples) + '_' \
                  + str(complete_n_samples) + 'pts_' + loss + '_' + experiment_tag

top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'

complete_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/raw', \
                             str(complete_n_samples), class_syn_id)


n_input = [incomplete_n_samples, 3]
n_output = [complete_n_samples, 3]

train_dir = osp.join(top_data_dir, 'OUT/iccv/nn_models/chair_2048_4096pts_chamfer_original')
#train_dir = osp.join(train_dir, experiment_name)
#create_dir(train_dir)

max_evaluation_epochs = max_training_epochs

Give me the class type: chair
Give an experiment tag-IDorigin


In [6]:
partial_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Partial_PCs/Angela_synthetic','2048')

In [7]:
seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

In [8]:
cmpl_file_names = pio.load_filenames_of_input_data(complete_pclouds_path, points_extension)
cmpl_pclouds, cmpl_model_names, cmpl_syn_ids = pio.load_crude_point_clouds(file_names=cmpl_file_names, n_threads=40)
assert(len(np.unique(cmpl_syn_ids)) == 1)
print '%d files containing complete point clouds were found.' % (len(cmpl_pclouds), )

6778 files containing complete point clouds were found.


In [9]:
test_list = '/orions4-zfs/projects/lins2/Panos_Space/DATA/Point_Clouds/Partial_PCs/Angela_synthetic/train_test_splits/test_models_small_with_scan_id.txt'
test_pclouds, test_model_ids, test_syn_ids = load_partial_pointclouds(test_list, partial_pclouds_path, 
                                                                               50, class_restriction=class_syn_id)
test_model_names = np.array([i[0] for i in test_model_ids], dtype=object)
test_scan_ids = np.array([i[1] for i in test_model_ids], dtype=object)
test_unames = np.array(test_model_names + '.' + test_scan_ids + '.' + test_syn_ids, dtype=object)
matching_index = indices_in_iterable(cmpl_model_names, test_model_names)
assert(np.alltrue(matching_index != -1))
test_data = PointCloudDataSet(cmpl_pclouds[matching_index], noise=test_pclouds, labels=test_unames)

6000 partial point clouds were loaded.


In [1]:
if load_model:
    print(osp.join('/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iccv/nn_models/chair_2048_4096pts_chamfer_original', 'configuration'))
    conf = Conf.load(osp.join('/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iccv/nn_models/chair_2048_4096pts_chamfer_original', 'configuration'))
else:    
    encoder_args = {'n_filters': [64, 128, 512],
                    'filter_sizes': [1, 1, 1],
                    'strides': [1, 1, 1],
                    'dropout_prob': 0.5
                   }
    
    decoder_args = {'layer_sizes': [1024, np.prod(n_output)]}
    
    conf = Conf(
                n_input = n_input,
                n_output = n_output,
                denoising = True,
                training_epochs = 1,#max_training_epochs,
                batch_size = 50,
                loss = loss,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 1,
                learning_rate = 0.0005,
                z_rotate = z_rotate,
                encoder = enc_dec.encoder_with_convs_and_symmetry,
                encoder_args = encoder_args,
                decoder = enc_dec.decoder_with_fc_only,
                decoder_args = decoder_args
               )
    
    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))
    
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)

if load_model:
    saved_epochs = read_saved_epochs(conf.train_dir)
    last_epoch = 56 #saved_epochs[-1]
    ae.restore_model(conf.train_dir,last_epoch, verbose=True)

NameError: name 'load_model' is not defined